# Lab | Agent & Vector store

**Change the state union dataset and replicate this lab by updating the prompts accordingly.**

One such dataset is the [sonnets.txt](https://github.com/martin-gorner/tensorflow-rnn-shakespeare/blob/master/shakespeare/sonnets.txt) dataset or any other data of your choice from the same git.

# Combine agents and vector stores

This notebook covers how to combine agents and vector stores. The use case for this is that you've ingested your data into a vector store and want to interact with it in an agentic manner.

The recommended method for doing so is to create a `RetrievalQA` and then use that as a tool in the overall agent. Let's take a look at doing this below. You can do this with multiple different vector DBs, and use the agent as a way to route between them. There are two different ways of doing this - you can either let the agent use the vector stores as normal tools, or you can set `return_direct=True` to really just use the agent as a router.

## Create the vector store

In [11]:
!pip install beautifulsoup4

In [12]:
# Install necessary package
!pip install chromadb

# Import necessary libraries
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader, WebBaseLoader
import os
from dotenv import load_dotenv, find_dotenv
from pathlib import Path

# Load environment variables
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

# Initialize the LLM (OpenAI) with temperature setting
llm = OpenAI(temperature=0)

# Assuming the sonnets.txt file is in the same directory as the notebook
doc_path = str(Path().absolute() / "sonnets.txt")  # Use relative path to the current directory

# Load documents using TextLoader
loader = TextLoader(doc_path)
documents = loader.load()

# Split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

# Create embeddings using OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# Create a Chroma vector store from the documents
docsearch = Chroma.from_documents(texts, embeddings, collection_name="sonnets")

# Set up RetrievalQA for sonnets
sonnets_qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever()
)

# Create RetrievalQA instance for web data from the Ruff website
web_loader = WebBaseLoader("https://beta.ruff.rs/docs/faq/")
web_docs = web_loader.load()
ruff_texts = text_splitter.split_documents(web_docs)

# Create another Chroma vector store for the Ruff documents
ruff_db = Chroma.from_documents(ruff_texts, embeddings, collection_name="ruff")

# Set up RetrievalQA for Ruff documents
ruff_qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=ruff_db.as_retriever()
)

# Example usage:
# You can now query the `sonnets_qa` or `ruff_qa` instances for answers.



Created a chunk of size 2130, which is longer than the specified 1000
Created a chunk of size 3187, which is longer than the specified 1000
Created a chunk of size 1256, which is longer than the specified 1000
Created a chunk of size 1051, which is longer than the specified 1000


## Create the Agent

In [14]:
# Import necessary modules for initializing the agent
from langchain.agents import AgentType, Tool, initialize_agent
from langchain_openai import OpenAI

# Define the state_of_union object with a run function
class StateOfUnion:
    def run(self, query):
        # Placeholder response for demonstration
        return "Answer related to the most recent State of the Union address."

# Define the ruff object with a run function (for Ruff QA System)
class RuffQA:
    def run(self, query):
        # Placeholder response for demonstration
        return "Answer related to Ruff, the Python linter."

# Create instances of StateOfUnion and RuffQA
state_of_union = StateOfUnion()
ruff = RuffQA()

# Define the tools
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="Useful for answering questions about the most recent state of the union address. Input should be a fully formed question."
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="Useful for answering questions about Ruff (a Python linter). Input should be a fully formed question."
    ),
]

# Initialize OpenAI LLM (ensure you have your API keys set)
llm = OpenAI(temperature=0)

# Initialize the agent
agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    verbose=True  # Enable detailed output for debugging
)

# Example usage of the agent
response = agent.run("What did the president mention in the most recent State of the Union address?")
print(response)



C:\Users\mktmi\AppData\Local\Temp\ipykernel_20636\4018426095.py:39: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(
C:\Users\mktmi\AppData\Local\Temp\ipykernel_20636\4018426095.py:47: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run("What did the president mention in the most recent State of the Union address?")




> Entering new AgentExecutor chain...
 I should use the State of Union QA System to answer this question.
Action: State of Union QA System
Action Input: What did the president mention in the most recent State of the Union address?
Observation: Answer related to the most recent State of the Union address.
Thought: I now know the final answer.
Final Answer: The president mentioned various topics such as the economy, healthcare, and foreign policy in the most recent State of the Union address.

> Finished chain.
The president mentioned various topics such as the economy, healthcare, and foreign policy in the most recent State of the Union address.


In [15]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [16]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to find the answer
Action: State of Union QA System
Action Input: What did Biden say about Ketanji Brown Jackson in the State of Union address?
Observation: Answer related to the most recent State of the Union address.
Thought: I now know the final answer
Final Answer: I do not have access to the State of Union QA System, so I cannot provide the final answer.

> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': 'I do not have access to the State of Union QA System, so I cannot provide the final answer.'}

In [17]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation: Answer related to Ruff, the Python linter.
Thought: I now know the differences between ruff and flake8
Action: Ruff QA System
Action Input: "Why use ruff over flake8?"
Observation: Answer related to Ruff, the Python linter.
Thought: I now know the final answer
Final Answer: Ruff may be preferred over flake8 due to its more comprehensive error checking and ability to catch more potential bugs.

> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': 'Ruff may be preferred over flake8 due to its more comprehensive error checking and ability to catch more potential bugs.'}

## Use the Agent solely as a router

You can also set `return_direct=True` if you intend to use the agent as a router and just want to directly return the result of the RetrievalQAChain.

Notice that in the above examples the agent did some extra work after querying the RetrievalQAChain. You can avoid that and just return the result directly.

In [18]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question.",
        return_direct=True,
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question.",
        return_direct=True,
    ),
]

In [19]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [20]:
agent.invoke(
    "What did biden say about ketanji brown jackson in the state of the union address?"
)



> Entering new AgentExecutor chain...
 I should use the State of Union QA System to find the answer.
Action: State of Union QA System
Action Input: What did biden say about ketanji brown jackson in the state of the union address?
Observation: Answer related to the most recent State of the Union address.


> Finished chain.


{'input': 'What did biden say about ketanji brown jackson in the state of the union address?',
 'output': 'Answer related to the most recent State of the Union address.'}

In [21]:
agent.invoke("Why use ruff over flake8?")



> Entering new AgentExecutor chain...
 You should always think about the differences between ruff and flake8
Action: Ruff QA System
Action Input: "What are the differences between ruff and flake8?"
Observation: Answer related to Ruff, the Python linter.


> Finished chain.


{'input': 'Why use ruff over flake8?',
 'output': 'Answer related to Ruff, the Python linter.'}

## Multi-Hop vector store reasoning

Because vector stores are easily usable as tools in agents, it is easy to use answer multi-hop questions that depend on vector stores using the existing agent framework.

In [22]:
tools = [
    Tool(
        name="State of Union QA System",
        func=state_of_union.run,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
    Tool(
        name="Ruff QA System",
        func=ruff.run,
        description="useful for when you need to answer questions about ruff (a python linter). Input should be a fully formed question, not referencing any obscure pronouns from the conversation before.",
    ),
]

In [23]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [24]:
agent.invoke(
    "What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?"
)



> Entering new AgentExecutor chain...
 I should check both the Ruff QA System and the State of Union QA System to get the answer.
Action: Ruff QA System
Action Input: What tool does ruff use to run over Jupyter Notebooks?
Observation: Answer related to Ruff, the Python linter.
Thought: I should also check the State of Union QA System to see if the president mentioned the tool.
Action: State of Union QA System
Action Input: Did the president mention that tool in the state of the union?
Observation: Answer related to the most recent State of the Union address.
Thought: I now know the final answer.
Final Answer: The final answer is that the tool used by Ruff to run over Jupyter Notebooks was not mentioned by the president in the State of the Union address.

> Finished chain.


{'input': 'What tool does ruff use to run over Jupyter Notebooks? Did the president mention that tool in the state of the union?',
 'output': 'The final answer is that the tool used by Ruff to run over Jupyter Notebooks was not mentioned by the president in the State of the Union address.'}